# Connect to MySQL DB with sqlalchemy
## with basic tutorial of sqlachemy

## References
- [Getting Started with SQLAlchemy](https://medium.com/geekculture/getting-started-with-sqlalchemy-d132d04c940)
- [Generating Random Data into a Database Using Python](https://towardsdatascience.com/generating-random-data-into-a-database-using-python-fd2f7d54024e)
- [SQLAlchemy — Python Tutorial](https://towardsdatascience.com/sqlalchemy-python-tutorial-79a577141a91)

# Load packages

In [1]:
%load_ext lab_black

In [2]:
import pandas as pd
from faker import Faker
from collections import defaultdict
import sqlalchemy as db
import pymysql
from _secrets import PASSWORD

# Connect MySQL DB

In [3]:
IP = "localhost"
DB_NAME = "testdb"
USER = "root"
db_url = f"mysql+pymysql://{USER}:{PASSWORD}@{IP}/{DB_NAME}"
engine = db.create_engine(db_url)

In [4]:
connection = engine.connect()
print(engine.table_names())

['test_table', 'user']


/home/ubuntu/.local/lib/python3.6/site-packages/ipykernel_launcher.py:2: SADeprecationWarning: The Engine.table_names() method is deprecated and will be removed in a future release.  Please refer to Inspector.get_table_names(). (deprecated since: 1.4)
  


# Generate dummy data & inject

In [5]:
fake = Faker()

In [6]:
fake_data = defaultdict(list)
for _ in range(1000):
    fake_data["first_name"].append(fake.first_name())
    fake_data["last_name"].append(fake.last_name())
    fake_data["occupation"].append(fake.job())
    fake_data["dob"].append(fake.date_of_birth())
    fake_data["country"].append(fake.country())
df_fake_data = pd.DataFrame(fake_data)

In [7]:
# Drop if table already exists
query = "DROP TABLE IF EXISTS user"
engine.execute(query)

# IF not, inject
df_fake_data.to_sql("user", con=engine, index=False)

# Fetch data from DB

In [8]:
metadata = db.MetaData()
user = db.Table("user", metadata, autoload=True, autoload_with=engine)
query = db.select(user)

In [9]:
result_proxy = connection.execute(query)
result_set = result_proxy.fetchall()
result_set[:5]

[('John', 'Peck', 'Mechanical engineer', datetime.date(1959, 10, 10), 'Ecuador'),
 ('Daniel', 'Harper', 'Producer, radio', datetime.date(1975, 7, 5), 'Wallis and Futuna'),
 ('Michelle', 'Berry', 'Herpetologist', datetime.date(1951, 4, 8), 'Hong Kong'),
 ('Mary', 'Murphy', 'Biochemist, clinical', datetime.date(1993, 3, 9), 'Georgia'),
 ('Michael', 'Schmidt', 'Occupational therapist', datetime.date(2019, 3, 6), 'Rwanda')]

# View table details

In [10]:
user.columns.keys()

['first_name', 'last_name', 'occupation', 'dob', 'country']

In [11]:
repr(metadata.tables["user"])

"Table('user', MetaData(), Column('first_name', TEXT(), table=<user>), Column('last_name', TEXT(), table=<user>), Column('occupation', TEXT(), table=<user>), Column('dob', DATE(), table=<user>), Column('country', TEXT(), table=<user>), schema=None)"

# Write query

## where

In [12]:
query = db.select([user]).where(user.columns.country == "Jamaica")
result_proxy = connection.execute(query)
result_set = result_proxy.fetchall()
result_set[:5]

[]

## in

In [13]:
query = db.select([user.columns.first_name, user.columns.country]).where(
    user.columns.country.in_(["Jamaica", "Mongolia"])
)
result_proxy = connection.execute(query)
result_set = result_proxy.fetchall()
result_set[:5]

[('Alicia', 'Mongolia'),
 ('Thomas', 'Mongolia'),
 ('Tammy', 'Mongolia'),
 ('Kathy', 'Mongolia'),
 ('Robert', 'Mongolia')]

## and, or, not

In [14]:
query = db.select([user.columns.first_name, user.columns.country]).where(
    user.columns.country.in_(["Jamaica", "Mongolia"]), user.columns.first_name != "Tony"
)
result_proxy = connection.execute(query)
result_set = result_proxy.fetchall()
result_set[:5]

[('Alicia', 'Mongolia'),
 ('Thomas', 'Mongolia'),
 ('Tammy', 'Mongolia'),
 ('Kathy', 'Mongolia'),
 ('Robert', 'Mongolia')]

## order by

In [15]:
query = db.select([user]).order_by(db.desc(user.columns.dob))
result_proxy = connection.execute(query)
result_set = result_proxy.fetchall()
result_set[:5]

[('Amy', 'Garcia', 'Biomedical engineer', datetime.date(2022, 2, 15), 'China'),
 ('Stephanie', 'Carrillo', 'Computer games developer', datetime.date(2022, 2, 2), 'Zambia'),
 ('Jennifer', 'Garcia', 'Financial planner', datetime.date(2021, 8, 11), 'Tuvalu'),
 ('Marie', 'Moore', 'Runner, broadcasting/film/video', datetime.date(2021, 5, 29), 'Malta'),
 ('David', 'White', 'Designer, industrial/product', datetime.date(2021, 1, 26), 'Hong Kong')]

## functions

In [16]:
query = db.select([db.func.count(user.columns.dob)])
result_proxy = connection.execute(query)
result_set = result_proxy.fetchall()
result_set[:5]

[(1000,)]

## group by

In [17]:
query = db.select([db.func.count(user.columns.country)]).group_by(user.columns.country)
result_proxy = connection.execute(query)
result_set = result_proxy.fetchall()
result_set[:5]

[(4,), (2,), (3,), (6,), (8,)]

## distinct
> The SQL DISTINCT clause is used to remove duplicates from the result set of a SELECT statement.

In [18]:
query = db.select([user.columns.country.distinct()])
result_proxy = connection.execute(query)
result_set = result_proxy.fetchall()
result_set[:5]

[('Ecuador',),
 ('Wallis and Futuna',),
 ('Hong Kong',),
 ('Georgia',),
 ('Rwanda',)]

# case & cast

In [19]:
"""
female_pop = db.func.sum(db.case([(census.columns.sex == 'F', census.columns.pop2000)],else_=0))
total_pop = db.cast(db.func.sum(census.columns.pop2000), db.Float)

query = db.select([female_pop/total_pop * 100])
result = connection.execute(query).scalar()
print(result)
"""

"\nfemale_pop = db.func.sum(db.case([(census.columns.sex == 'F', census.columns.pop2000)],else_=0))\ntotal_pop = db.cast(db.func.sum(census.columns.pop2000), db.Float)\n\nquery = db.select([female_pop/total_pop * 100])\nresult = connection.execute(query).scalar()\nprint(result)\n"